# IEOR 4501 Final Project: Understanding Hired Rides in NYC

#### Contributors: Joy Ren(jr4154), Yiwen Qian(yq2346)

## Project Setup

In [1]:
# all import statements needed for the project

import math
from math import sin, cos, sqrt, atan2, radians
import os
import warnings
import re
import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
import geopandas as gpd
from matplotlib.figure import Figure
import matplotlib.animation as animation
from IPython.display import HTML
from keplergl import KeplerGl
from matplotlib.backends.backend_agg import FigureCanvasAgg
warnings.filterwarnings('ignore')

In [2]:
# any constants need; some have been added for you, and 
# some you need to fill in

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_ZONES_DIR = "taxi_zones"
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
UBER_DATA = "uber_rides_sample.csv"
WEATHER_CSV_DIR = "weather"

CRS = 4326  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [3]:
# Make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise

## Part 1: Data Preprocessing
In Part 1, we downloaded Parquet files, cleaning and filtering for the relevant data, filling in missing data, and generating samples of these raw datasets.
Our processing stages can be illustrated as:
* 1. Load the Taxi Zones by x and y coordinates.
* 2. Calculate distance using latitude and longtitude.
* 3. Use `bs4` and `requests` module to parse html, then we can process Yellow Taxi data of NYC.
* 4. Processing Uber Data.
* 5. Processing Weather Data.
* 6. Save the cleaned data for further analyis. 

### Load Taxi Zones

In [4]:
def load_taxi_zones(shapefile):
    return gpd.read_file(shapefile)     #read shapefile

In [5]:
load_taxi_zones(TAXI_ZONES_SHAPEFILE)

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.770 256767.698, 1026495.593 ..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.310 144283.336, 936046.565 14..."
...,...,...,...,...,...,...,...
258,259,0.126750,0.000395,Woodlawn/Wakefield,259,Bronx,"POLYGON ((1025414.782 270986.139, 1025138.624 ..."
259,260,0.133514,0.000422,Woodside,260,Queens,"POLYGON ((1011466.966 216463.005, 1011545.889 ..."
260,261,0.027120,0.000034,World Trade Center,261,Manhattan,"POLYGON ((980555.204 196138.486, 980570.792 19..."
261,262,0.049064,0.000122,Yorkville East,262,Manhattan,"MULTIPOLYGON (((999804.795 224498.527, 999824...."


In [6]:
def lookup_coords_for_taxi_zone_id(zone_loc_id):
    zones= load_taxi_zones(TAXI_ZONES_SHAPEFILE)
    centroid= zones[zones['LocationID'] == zone_loc_id].centroid.values[0]
    return (centroid.y, centroid.x)        # return the centroid coordinates

### Calculate distance

In [7]:
def calculate_distance_with_coords(from_coord, to_coord):
    ## "from_coord": columns 'pickup_latitude' and 'pickup_longitude' from datarame;
    ## "to_coord":  columns 'dropoff_latitude' and 'dropoff_longitude' from datarame;
    
    R = 6373.0  # approximate radius of earth in km    
    lat1, lon1 = from_coord
    lat2, lon2 = to_coord

    dlat = radians(lat2-lat1)
    dlon = radians(lon2-lon1)
    
    ## Calculate the distance between two coordinates in kilometers using the Haversine formula
    a = sin(dlat/2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * (sin(dlon/2) ** 2 )
    coord_distance = 2 * R * atan2(math.sqrt(a), sqrt(1-a))
    
    return coord_distance

In [8]:
def calculate_distance_with_zones(from_zone, to_zone):
    ##Calculate the distance between two taxi zones in kilometers using the centroid of each zone
    
    from_zone_coord = lookup_coords_for_taxi_zone_id(from_zone)
    to_zone_coord = lookup_coords_for_taxi_zone_id(to_zone)   # get two coordinates from two zone ids
    
    return calculate_distance_with_coords(from_zone_coord, to_zone_coord)

In [9]:
def add_distance_column(dataframe):               
    distance=[]
    for index, row in dataframe.iterrows():
        from_coord=(row['pickup_latitude'],row['pickup_longitude'])
        to_coord=(row['dropoff_latitude'],row['dropoff_longitude'])
        distance.append(calculate_distance_with_coords(from_coord,to_coord))  
    dataframe= pd.concat([dataframe,pd.DataFrame(distance,columns=['distance'])], axis=1)
    return dataframe

### Process Taxi Data
Here, We want to obtain the Yellow Taxi Data January 2009 through June 2015, then do the cleaning and combining to dataframes.

In [10]:
def get_all_urls_from_taxi_page(taxi_page):
    # Get the HTML content of the page
    res = requests.get(taxi_page)
    soup = bs4.BeautifulSoup(res.content, 'html.parser')
    # Find all <a> tags with href attribute
    urls = [a['href'] for a in soup.find_all('a', href=re.compile(".*(2009|201[0-4]|2015-0[1-6]).*"))]
    return urls

In [11]:
all_urls=get_all_urls_from_taxi_page(TAXI_URL)
all_urls

['https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2015-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2015-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2015-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2015-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2015-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2015-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-04.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2015-04.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2015-04.parquet',
 'https:

In [12]:
def filter_taxi_parquet_urls(all_urls):
    """
    Given a list of URLs, filters out the URLs that do not contain yellow taxi data parquet files.
    """
    parquet_urls = []
    for url in all_urls:
        if 'yellow_tripdata' in url:
            parquet_urls.append(url)
    return parquet_urls

In [13]:
def get_and_clean_month(url):
    """
    clean the data, and return a pandas DataFrame.
    """          

    df = pd.read_parquet(url)
    df = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'fare_amount']]
    df = df.dropna()


    
    zone=load_taxi_zones(TAXI_ZONES_SHAPEFILE)
    zone = zone.to_crs(4326)
    zone['latitude'] = zone.geometry.centroid.y
    zone['longitude'] = zone.geometry.centroid.x
    

    # remove unnecessary columns 
    # create mapping methods for latitude & longitude
    # aim to create lat & longitude which do not exist in some taxi data
    zone = zone[['LocationID', 'longitude', 'latitude', 'zone', 'borough']]
    lat_map = dict(zip(zone['LocationID'], zone['latitude']))
    lon_map = dict(zip(zone['LocationID'], zone['longitude']))
    
    
    # process dataframe which only has LocationIDs 
    # match the LocationID in 'df' with dataframe 'g'
    # add 4 new columns to 'df' after matching
    if 'DOLocationID' in df:
        df['pickup_latitude']  = df['PULocationID'].map(lat_map)
        df['pickup_longitude'] = df['PULocationID'].map(lon_map)
        df['dropoff_latitude'] = df['DOLocationID'].map(lat_map)
        df['dropoff_longitude']= df['DOLocationID'].map(lon_map)
        
     # remove trips if start and/or end outside New York area
    df = df[df['pickup_latitude'] >= NEW_YORK_BOX_COORDS[0][0]]
    df = df[df['pickup_latitude'] <= NEW_YORK_BOX_COORDS[1][0]]
    df = df[df['pickup_longitude'] >= NEW_YORK_BOX_COORDS[0][1]]
    df = df[df['pickup_longitude'] <= NEW_YORK_BOX_COORDS[1][1]]
        
    df = df[df['dropoff_latitude'] >= NEW_YORK_BOX_COORDS[0][0]]
    df = df[df['dropoff_latitude'] <= NEW_YORK_BOX_COORDS[1][0]]
    df = df[df['dropoff_longitude'] >= NEW_YORK_BOX_COORDS[0][1]]
    df = df[df['dropoff_longitude'] <= NEW_YORK_BOX_COORDS[1][1]]
    
    # normalize column names
    df = df.rename(columns={'tpep_pickup_datetime':'pickup_datetime', 
                            'Trip_Pickup_DateTime':'pickup_datetime', 
                            'tpep_dropoff_datetime':'dropoff_datetime',
                            'Fare_Amt':'fare_amount'         
                            })
        
    
    df = df[(df['fare_amount'] >0)]     
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
    df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')
    
    
    
    #df['distance_km'] = df.apply(lambda row: calculate_distance_with_zones(row['PULocationID'], row['DOLocationID']), axis=1)
    return df

In [14]:
def get_and_clean_taxi_data(parquet_urls):
    all_taxi_dataframes = []
    
    for parquet_url in parquet_urls:
        print(f"working on {parquet_url}")
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        if os.path.exists("somefilename..." ):
            dataframe.from_csv("somefilename")
        else:
            dataframe = get_and_clean_month(parquet_url)
            dataframe.to_csv("soemfilename...")
    
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        add_distance_column(dataframe)
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes)
    return taxi_data

In [15]:
def get_taxi_data():
    # download files to the directory "yellow taxi" if files do not exist
    all_urls = get_all_urls_from_taxi_page(TAXI_URL)
    all_parquet_urls = filter_taxi_parquet_urls(all_urls)
    all_files = []
    for url in all_parquet_urls:
        print(f"working on {url}")
        response = requests.get(url, stream=True)
        file_name = url.split("/")[-1]
        local_file_path = os.path.join("yellow taxi", file_name)
        all_files.append(local_file_path)
        
        if not os.path.exists(local_file_path):
            with open(local_file_path, "wb") as file:
                file.write(response.content)
            print(f"done with {url}")
    
#     print(all_files)
    taxi_data = get_and_clean_taxi_data(all_files)
    return taxi_data

In [16]:
taxi_data = get_taxi_data()

working on https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-01.parquet
working on https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-02.parquet
working on https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-03.parquet
working on https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-04.parquet
working on https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-05.parquet
working on https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-06.parquet
working on https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-01.parquet
working on https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-02.parquet
working on https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-03.parquet
working on https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-04.parquet
working on https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-05.parquet

working on yellow taxi/yellow_tripdata_2013-03.parquet
working on yellow taxi/yellow_tripdata_2013-04.parquet
working on yellow taxi/yellow_tripdata_2013-05.parquet
working on yellow taxi/yellow_tripdata_2013-06.parquet
working on yellow taxi/yellow_tripdata_2013-07.parquet
working on yellow taxi/yellow_tripdata_2013-08.parquet
working on yellow taxi/yellow_tripdata_2013-09.parquet
working on yellow taxi/yellow_tripdata_2013-10.parquet
working on yellow taxi/yellow_tripdata_2013-11.parquet
working on yellow taxi/yellow_tripdata_2013-12.parquet
working on yellow taxi/yellow_tripdata_2012-01.parquet
working on yellow taxi/yellow_tripdata_2012-02.parquet
working on yellow taxi/yellow_tripdata_2012-03.parquet
working on yellow taxi/yellow_tripdata_2012-04.parquet
working on yellow taxi/yellow_tripdata_2012-05.parquet
working on yellow taxi/yellow_tripdata_2012-06.parquet
working on yellow taxi/yellow_tripdata_2012-07.parquet
working on yellow taxi/yellow_tripdata_2012-08.parquet
working on

KeyError: "['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID'] not in index"

In [ ]:
taxi_data.head()

### Processing Uber Data

In [ ]:
def load_and_clean_uber_data(csv_file):
    df = pd.read_csv(csv_file)
    df = df.dropna(how='any')      #remove missing values
    df = df.drop(columns=['Unnamed: 0','key','passenger_count'])      #remove unnecessary columns
    df = df.loc[df.pickup_longitude.between(-74.242330,-73.717047) & df.dropoff_longitude.between(-74.242330,-73.717047) 
         & df.pickup_latitude.between(40.560445,40.908524)& df.dropoff_latitude.between(40.560445,40.908524)]
          # remove locations out of range
    df.reset_index(drop=True, inplace=True)    #reset index
    return df

In [ ]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_DATA)
    uber_dataframe = add_distance_column(uber_dataframe)
    uber_dataframe["pickup_datetime"] = pd.to_datetime(uber_dataframe['pickup_datetime'])
    uber_dataframe["year"] = uber_dataframe["pickup_datetime"].dt.year
    uber_dataframe["month"] = uber_dataframe["pickup_datetime"].dt.month
    uber_dataframe["day"] = uber_dataframe["pickup_datetime"].dt.day
    uber_dataframe["hour"] = uber_dataframe["pickup_datetime"].dt.hour
    uber_dataframe["dayofweek"] = uber_dataframe["pickup_datetime"].dt.dayofweek
    return uber_dataframe

In [ ]:
uber_data = get_uber_data()

In [ ]:
uber_data.head()

### Processing Weather Data

In [ ]:
def get_all_weather_csvs(directory):
    csv_files = [file for file in os.listdir(directory) if file.endswith(".csv")]
    return csv_files

In [ ]:
# get all the "weather" csv files from 2009-2015
get_all_weather_csvs(WEATHER_CSV_DIR)

In [ ]:
def clean_month_weather_data_hourly(csv_file):
    df = pd.read_csv(csv_file)
    
    # remove columns that have all NaN values
    df=df.dropna(axis=1, how="all")
    
    # for "hourly wind speed", fill missing value (calculate hourly-data from daily-data)
    if df['HourlyWindSpeed'] is 'NaN':
        df['HourlyWindSpeed'] = df['DailyAverageWindSpeed']/24
        df['HourlyWindSpeed'].apply(lambda x: round(x,2))
        
    # only keep necessary columns
    keep = ['DATE', 'HourlyPrecipitation', 'HourlyWindSpeed']
    
    #in "hourly precipation" column, replace "T" with value 0, replace NaN with 0 since existing value 
    # for the column is small
    df['HourlyPrecipitation'] = df['HourlyPrecipitation'].replace({pd.np.nan: 0, 'T': 0})
    df['HourlyPrecipitation'] = df['HourlyPrecipitation'].apply(  lambda x: x.replace('s', '') 
                                                                if isinstance(x, str) else x)
    df['HourlyPrecipitation'] = df['HourlyPrecipitation'].astype(float)
    
    df1 = pd.DataFrame()
    for i in keep:
        col1=df.loc[:,df.columns.str.contains(i)]
        df1=pd.concat([df1, col1], axis=1)
    hourly_clean= df1.dropna()
    hourly_clean.reset_index(drop=True, inplace=True)
    
    # Add columns of year, month, day and hour
    hourly_clean["DATE"] = pd.to_datetime(hourly_clean['DATE'])
    hourly_clean["YEAR"] = hourly_clean["DATE"].dt.year
    hourly_clean["MONTH"] = hourly_clean["DATE"].dt.month
    hourly_clean["DAY"] = hourly_clean["DATE"].dt.day
    hourly_clean["HOUR"] = hourly_clean["DATE"].dt.hour
    hourly_clean['HourlyPrecipitation'] = hourly_clean['HourlyPrecipitation'].round(2)
    hourly_clean.dropna(inplace=True)
    return hourly_clean

In [ ]:
clean_month_weather_data_hourly('weather/2012_weather.csv')

In [ ]:
def clean_month_weather_data_daily(csv_file):
    df = pd.read_csv(csv_file)
    
    #remove columns having all NaN values
    df = df.dropna(axis=1, how="all")
    
    #in "hourly precipation" column, replace "T" with value 0, replace NaN with 0 since existing value 
    # for the column is small
    df['HourlyPrecipitation'] = df['HourlyPrecipitation'].replace({pd.np.nan: 0, 'T': 0})
    df['HourlyPrecipitation'] = df['HourlyPrecipitation'].apply(  lambda x: x.replace('s', '') 
                                                                if isinstance(x, str) else x)
    df['HourlyPrecipitation'] = df['HourlyPrecipitation'].astype(float)
    
    df['DATE'] = pd.to_datetime(df['DATE']) 
    df['DATE'] = df['DATE'].apply(lambda x: x.date())
    
    # Group by same date, calculate the mean,use that number as daily data
    Daily_w = df.groupby('DATE')['HourlyWindSpeed'].mean()
    Daily_p = df.groupby('DATE')['HourlyPrecipitation'].mean()
    #df1 = df.drop_duplicates(subset=['DATE'])
    df = df[['DATE']]
    df = pd.merge(df, Daily_w, on='DATE')
    df= pd.merge(df, Daily_p, on='DATE')
    
    df.rename(columns={'HourlyWindSpeed':'DailyWindSpeed','HourlyPrecipitation':'DailyPrecipitation'},inplace=True)
    
    df['DailyWindSpeed'] = df['DailyWindSpeed'].round(2)
    df['DailyPrecipitation'] = df['DailyPrecipitation'].round(2)
    
    # Add columns of year, month, day and hour
    df["DATE"] = pd.to_datetime(df['DATE'])
    df["YEAR"] = df["DATE"].dt.year
    df["MONTH"] = df["DATE"].dt.month
    df["DAY"] = df["DATE"].dt.day
    df["HOUR"] = df["DATE"].dt.hour
    
    return df

In [ ]:
clean_month_weather_data_daily('weather/2012_weather.csv')

#### Extra_credit for sunrise dataset:

In [ ]:
def clean_weather_data_dailysunrise(csv_file):
    df = pd.read_csv(csv_file)

    titles = ['DATE', 'Sunrise', 'Sunset']
    df_sunrise = pd.DataFrame()
    for i in titles:
        colms = df.loc[:,df.columns.str.contains(i)]
        df_sunrise = pd.concat([df_sunrise, colms], axis=1)
    sunrise_clean = df_sunrise.dropna()
    sunrise_clean.reset_index(drop=True, inplace=True)
    # Add the columns of year, month, day and hour according to the date
    sunrise_clean["DATE"] = pd.to_datetime(sunrise_clean['DATE'])
    sunrise_clean["YEAR"] = sunrise_clean["DATE"].dt.year
    sunrise_clean["MONTH"] = sunrise_clean["DATE"].dt.month
    sunrise_clean["DAY"] = sunrise_clean["DATE"].dt.day
    sunrise_clean["HOUR"] = sunrise_clean["DATE"].dt.hour
    # Add the columns of hour and minutes according to the sunrise time
    sunrise_clean["Sunrise"] = pd.to_datetime(sunrise_clean["Sunrise"],format='%H%M')
    sunrise_clean["Sunrise_hour"] = sunrise_clean["Sunrise"].dt.hour
    sunrise_clean["Sunrise_minute"] = sunrise_clean["Sunrise"].dt.minute
    # Add the columns of hour and minutes according to the sunset time
    sunrise_clean["Sunset"] = pd.to_datetime(sunrise_clean["Sunset"],format='%H%M')
    sunrise_clean["Sunset_hour"] = sunrise_clean["Sunset"].dt.hour
    sunrise_clean["Sunset_minute"] = sunrise_clean["Sunset"].dt.minute
    return sunrise_clean

In [ ]:
def load_and_clean_weather_data():
    weather_csv_files = get_all_weather_csvs(WEATHER_CSV_DIR)
    
    hourly_dataframes = []
    daily_dataframes = []
    sunrise_dataframes = []
        
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(WEATHER_CSV_DIR+"/"+csv_file)
        daily_dataframe = clean_month_weather_data_daily(WEATHER_CSV_DIR+"/"+csv_file)
        sunrise_dataframe = clean_weather_data_dailysunrise(WEATHER_CSV_DIR+"/"+csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        sunrise_dataframes.append(sunrise_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    sunrise_data = pd.concat(sunrise_dataframes)
    
    return hourly_data, daily_data, sunrise_data

In [ ]:
hourly_weather_data, daily_weather_data, sunrise_data = load_and_clean_weather_data()

In [ ]:
hourly_weather_data.head()

In [ ]:
daily_weather_data.head()

In [ ]:
sunrise_data.head()

## Part 2: Storing Cleaned Data

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather
(
    id INTEGER PRIMARY KEY,
    DATE DATE,
    HourlyPrecipitation FLOAT,
    HourlyWindSpeed FLOAT,
    YEAR INTEGER,
    MONTH INTEGER,
    DAY INTEGER,
    HOUR INTEGER
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather
(
    id INTEGER PRIMARY KEY,
    DATE DATE,
    DailyWindSpeed FLOAT,
    DailyPrecipitation FLOAT,
    YEAR INTEGER,
    MONTH INTEGER,
    DAY INTEGER,
    HOUR INTEGER
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips (
    id INTEGER PRIMARY KEY,
    pickup_datetime DATE,
    dropoff_datetime DATE,
    PULocationID INTEGER,
    DOLocationID INTEGER,
    fare_amount FLOAT,
    pickup_latitude FLOAT,
    pickup_longitude FLOAT,
    dropoff_latitude FLOAT,
    dropoff_longitude FLOAT,
    distance FLOAT
);
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips
(
    id INTEGER PRIMARY KEY,
    fare_amount FLOAT,
    pickup_datetime DATETIME,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    distance FLOAT,
    year INTEGER,
    month INTEGER,
    day INTEGER,
    hour INTEGER,
    dayofweek INTEGER
);
"""

SUNRISE_SCHEMA = """
CREATE TABLE IF NOT EXISTS sunrise_data
(
    id INTEGER PRIMARY KEY,
    DATE TEXT,
    Sunrise TEXT,
    Sunset TEXT,
    YEAR INTEGER,
    MONTH INTEGER,
    DAY INTEGER,
    HOUR INTEGER,
    Sunrise_Hour INTEGER,
    Sunrise_Minute INTEGER,
    Sunset_Hour INTEGER,
    Sunset_Minute INTEGER
)
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)
    f.write(SUNRISE_SCHEMA)

In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    afile = open(DATABASE_SCHEMA_FILE, 'r')
    sqlFile = afile.read()
    afile.close()

    sqlQuery = sqlFile.split(';')
    for query in sqlQuery:
        connection.execute(query)

### Add Data to Database

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    # convert all pd DateFrames to sql tables
    for table, df in table_to_df_dict.items():
        df.to_sql(table, engine,if_exists='append', index=False)

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_weather_data,
    "daily_weather": daily_weather_data,
    "sunrise_data": sunrise_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    with open(QUERY_DIRECTORY + outfile, "w") as f:
        f.write(query)

### Query 1:For 01-2009 through 06-2015, show the popularity of Yellow Taxi rides for each hour of the day. The query result should have 24 bins, one for each hour, descending.


In [ ]:
QUERY_1_FILENAME = "Hourly_popularity_Yellow_Taxi.sql"

QUERY_1 = """
SELECT COUNT(id) AS number,
strftime('%H', pickup_datetime) AS time
FROM taxi_trips
WHERE pickup_datetime BETWEEN '2009-01-01' AND '2015-06-31'
GROUP BY time
ORDER BY number DESC
"""

In [ ]:
engine.execute(QUERY_1).fetchall()

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

### Query 2: For the same time frame, show the popularity of Uber rides for each day of the week.The result should have 7 bins, one for each day, descending.

In [ ]:
QUERY_2_FILENAME = "Day_popularity_Uber.sql"

QUERY_2 = """
SELECT COUNT(id) AS number,
strftime('%w', pickup_datetime) AS time
FROM uber_trips
WHERE pickup_datetime BETWEEN '2009-01-01' AND '2015-06-31'
GROUP BY time
ORDER BY number DESC
"""

In [ ]:
engine.execute(QUERY_2).fetchall()

In [ ]:
write_query_to_file(QUERY_2, QUERY_2_FILENAME)

### Query 3: What is the 95% percentile of distance traveled for all hired trips during July 2013? The result should be a float. It’s okay if it’s a single float within a list and/or tuple.


In [ ]:
QUERY_3_FILENAME = "July_2013_95percent_distances.sql"

QUERY_3 = """
WITH all_trips AS(
    SELECT pickup_datetime, distance
    FROM taxi_trips
    WHERE pickup_datetime BETWEEN '2013-07-01' AND '2013-08-01'
    UNION ALL
    SELECT pickup_datetime, distance
    FROM uber_trips
    WHERE pickup_datetime BETWEEN '2013-07-01' AND '2013-08-01'
)
SELECT distance
FROM all_trips
ORDER BY distance 
LIMIT 1
OFFSET (SELECT COUNT(*) FROM all_trips) * 95 / 100 - 1
"""

In [ ]:
engine.execute(QUERY_3).fetchall()

In [ ]:
write_query_to_file(QUERY_3, QUERY_3_FILENAME)

### Query 4:  What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day? 

#### The result should be a list of 10 tuples. Each tuple should have three items: a date, an integer for the number of rides of that date, and a float for the average distance of that date. The list of tuples should be sorted by total number of rides, descending.

In [ ]:
QUERY_4_FILENAME = "2009_top_10_days.sql"

QUERY_4 = """
WITH all_trips AS (
    SELECT pickup_datetime, distance
    FROM taxi_trips
    WHERE pickup_datetime BETWEEN '2009-01-01' AND '2010-01-01'
    UNION ALL
    SELECT pickup_datetime, distance
    FROM uber_trips
    WHERE pickup_datetime BETWEEN '2009-01-01' AND '2010-01-01'
)
SELECT DATE(pickup_datetime) AS day,
AVG(distance) AS average_dis,
COUNT(*) AS number
FROM all_trips
GROUP BY DATE(pickup_datetime)
ORDER BY number DESC
LIMIT 10
"""

In [ ]:
engine.execute(QUERY_4).fetchall()

In [ ]:
write_query_to_file(QUERY_4, QUERY_4_FILENAME)

### Query 5: Which 10 days in 2014 were the windiest on average, and how many hired trips were made on those days? 

#### The result should be a list of 10 tuples. Each tuple should have three items: a date, a float for the average wind speed of that day, and the number of hired trips for that day. The list of tuples should be sorted by the average wind speed, descending.

In [ ]:
QUERY_5_FILENAME = "2014_top_10_windiest_days.sql"

QUERY_5 = """
WITH all_trips AS (
    SELECT DATE(pickup_datetime) AS pickup_date, 
    distance
    FROM taxi_trips
    UNION ALL
    SELECT DATE(pickup_datetime) AS pickup_date, 
    distance
    FROM uber_trips
),

windiest_days AS (
    SELECT DATE(DATE) AS wind_date,
    DailyWindSpeed
    FROM daily_weather
    WHERE wind_date BETWEEN '2014-01-01' AND '2015-01-01'
    ORDER BY DailyWindSpeed DESC
    LIMIT 10
)

SELECT w.wind_date,
COUNT(*)
FROM windiest_days AS w,all_trips AS a
WHERE w.wind_date == a.pickup_date
GROUP BY w.wind_date
"""

In [ ]:
engine.execute(QUERY_5).fetchall()

In [ ]:
write_query_to_file(QUERY_5, QUERY_5_FILENAME)

### Query 6: During Hurricane Sandy in NYC (Oct 29-30, 2012), plus the week leading up and the week after, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed? 


#### The result should be a list of roughly 384 tuples, where each tuple is an entry for every single hour of the given date range, even if no rides were taken, no precipitation was measured, or there was no wind. Each tuple should have four items: a string for the date and hour, an int for the number of hired rides in that hour, the float for the total precipitation for that hour, and a float for the average wind speed for that hour. The list of tuples should be ordered by date+hour, ascending.


In [ ]:
QUERY_6_FILENAME = "Hurricane_Sandy_in_NYC.sql"

QUERY_6 = """
WITH hourly_data AS (
SELECT strftime ('%Y %m %d %H', DATE) AS date_hour, 
AVG(HourlyWindSpeed) AS hourly_wind_speed, 
AVG(HourlyPrecipitation) AS hourly_precipitation 
FROM hourly_weather
WHERE DATE BETWEEN '2012-10-22' AND '2012-11-05'
GROUP BY date_hour ),

hired_trips AS (
SELECT strftime ('%Y %m %d %H', pickup_datetime) AS pick_up_date_hour,
    distance
    FROM taxi_trips
    WHERE pickup_datetime BETWEEN '2012-10-22' AND '2012-11-05'
    UNION ALL
    SELECT strftime ('%Y %m %d %H', pickup_datetime) AS pick_up_date_hour,
    distance
    FROM uber_trips
    WHERE pickup_datetime BETWEEN '2012-10-22' AND '2012-11-05'
)

SELECT hd.date_hour as date, 
COALESCE(COUNT(ht.pick_up_date_hour),0) AS num, 
hourly_wind_speed, 
hourly_precipitation
FROM hourly_data AS hd
LEFT JOIN hired_trips AS ht
ON hd.date_hour = ht.pick_up_date_hour
GROUP BY hd.date_hour
"""

In [ ]:
engine.execute(QUERY_6).fetchall()

In [ ]:
write_query_to_file(QUERY_6, QUERY_6_FILENAME)

#### Extra_credit : Add an additional SQL table that contains daily sunset/sunrise data (which can be found in the original weather data). Write a question that considers this new table in relation to one or more existing tables, and then create either a SQL query or a visualization that answers that question.

##### Question: Compare the difference of tips for trips that before and after sunset 

In [ ]:
#Join taxi data and sunset data using the year, month and day
query = """
        SELECT Sunrise_hour +1 AS Sunrise_hour,
        Sunrise_minute,
        t.hour +1 AS Pickup_Hour,
        t.minute AS Pickup_minute, 
        tip
        FROM sunrise_data AS s
        JOIN taxi_trips AS t 
        ON s.month = t.month AND s.day = t.day AND s.year = t.year 
        """
sunrise1 = pd.read_sql(query, engine)
sunrise.dropna()
#filter out the trips after sunrise
sunrise2 = sunrise1.loc[sunrise1["Pickup_Hour"] >= sunrise1["Sunrise_hour"]]
after_sunrise = sunrise2.loc[sunrise2["Pickup_minute"] >= sunrise2["Sunrise_minute"]]
#drop the trips that after sunrise using index values and we get trips before sunrise
after = after_sunrise.index
before_sunrise = sunrise1.drop(after)
#compare the average tips amount for trips that before and after sunrise
def after_sunrise_tip_higher():
    if before_sunset["tip"].mean() < after_sunset["tip"].mean():
        return True
    return False

after_sunrise_tip_higher()

#### Conclusion:

## Part 4: Visualizing the Data

### Visualization 1

### Create an appropriate visualization for the first query/question in part 3: For 01-2009 through 06-2015, show the popularity of Yellow Taxi rides for each hour of the day.

In [ ]:
def plot_visual_1(df):
    fig = plt.figure(figsize=(8, 6))
    x,y = df['time'],df['num']
    plt.bar(x,y, label='bar')
    plt.xlabel('Time in a day')
    plt.ylabel('Number of yellow taxi rides')
    plt.title('popularity of Yellow Taxi rides for each hour of the day from 01-2009 to 06-2015')    
    plt.legend(frameon=True, loc="upper left")
    plt.show()

In [ ]:
def get_data_for_visual_1():
    df = pd.read_sql_query(QUERY_1, engine)
    df = df.sort_values('time', ascending=True)
    return df

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)

#### Extra_credit: Animation for V1

In [ ]:
def get_img_list():
    df = get_data_for_visual_1()
    img_list = []
    for i in range(1,24):
        fig = Figure(figsize=(10,4))
        canvas = FigureCanvasAgg(fig)
        dfCurr = df.iloc[:i, :]
        x, y = dfCurr['time'], dfCurr['num']
        ax = fig.subplots(1,1)
        ax.bar(x, y, label='bar')
        ax.set_xlabel('time')
        ax.set_ylabel('Number')
        ax.set_title('Number of Taxi Trips each hour')    
        ax.legend(frameon=True, loc="upper left")
        canvas.draw()
        rgba = np.asarray(canvas.buffer_rgba())
        img_list.append(rgba)
    return img_list

In [ ]:
# make it an animation
img_list = get_img_list()
print(len(img_list))
vidname = "visual 1 vid"
fps = 16
fig = plt.figure(figsize=(15,10))
plt.axis("off")
ims = [[plt.imshow(i, animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=300, repeat_delay=1000,blit=True) 
# ani.save(vidname+".mp4",fps=16)
HTML(ani.to_jshtml())

### Visualization 2

### Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month) for both taxis and Ubers combined. Include the 90% confidence interval around the mean in the visualization.


In [ ]:
def plot_visual_2(dataframe): 
    
    # plot lines for taxi and uber
    plt.plot(t['month'], t['avg_dis'], label='Taxi', color='red')
    plt.plot(u['month'], u['avg_dis'], label='Uber', color='blue')
    
    # calculate 90% CI    
    plt.fill_between(t['month'], t['avg_dis']-1.645*taxi_std, taxi['avg_dis']+1.645*taxi_std, color='red', alpha=0.1)
    plt.fill_between(u['month'], u['avg_dis']-1.645*uber_std, uber['avg_dis']+1.645*uber_std, color='blue', alpha=0.1)

    # Set x-axis, y-axis label and title
    plt.xlabel('Month')
    plt.ylabel('Average Distance Travelled')
    plt.title('Average distance travelled per month with 90% CI')
    plt.show()

In [ ]:
def get_data_for_visual_2():
    
    taxi="""SELECT strftime ('%m', pickup_datetime) AS month, AVG(distance) as avg_dis
              FROM taxi_trips
              GROUP BY month
           """
    uber="""SELECT strftime ('%m', pickup_datetime) AS month, AVG(distance) as avg_dis
              FROM uber_trips
              GROUP BY month
           """
    avg_taxi_distance = pd.read_sql_query(taxi, engine)
    avg_uber_distance = pd.read_sql_query(uber, engine)
    return avg_taxi_distance,avg_uber_distance

In [ ]:
t,u = get_data_for_visual_2()
plot_visual_2(t,u)

### Visualization 3

### Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.  

In [ ]:
def get_data_for_visual_3():
   
    boxes = {
        "LGA": [-73.889473,40.766102,-73.857630,40.782806],
        "JFK" : [-73.826978,40.618945,-73.741319,40.673388],
        "EWR" : [-74.199343,40.668791,-74.150248,40.712069]}
    df = pd.DataFrame(index=range(7))
    
    for (addr, box) in boxes.items():
        query=f"""
        WITH data AS
        (SELECT pickup_datetime, dropoff_latitude, dropoff_longitude FROM uber_trips 
        UNION ALL
        SELECT pickup_datetime, dropoff_latitude, dropoff_longitude FROM taxi_trips)
        
        SELECT count(*) AS {addr}, strftime('%w', pickup_datetime) AS `day_of_week` 
        FROM data WHERE dropoff_longitude>={bbox[0]} and dropoff_latitude>={bbox[1]}
        and dropoff_longitude<={bbox[2]} and dropoff_latitude<={bbox[3]} 
        GROUP BY `day_of_week`;
        """
        dt = pd.read_sql_query(query, engine)
        df = pd.concat([df, dt.iloc[:, 0]], axis=1)
    df = pd.DataFrame(df)
    df.index = df.index.map(lambda x: x+1)    
    return df

In [ ]:
df = get_data_for_visual_3()

plt.plot(df.index, df['LGA'], label='LGA')
plt.plot(df.index, df['JFK'], label='JFK')
plt.plot(df.index, df['EWR'], label='EWR')
plt.legend()
plt.xlabel('Day of week')
plt.ylabel('Number of dropoffs')
plt.show()

### Visualization 4
### Create a heatmap of all hired trips over a map of the area. Consider using Folium or KeplerGL or another library that helps generate geospatial visualizations.

In [ ]:
def get_data_for_visual_4():
    
    g = gpd.read_file(filename='taxi_zones.zip', engine='fiona')
    g = g.to_crs(4326)
    g['longitude'] = g.centroid.x
    g['latitude'] = g.centroid.y
    
    # write queries to fetch latitude and longitude from trips
    query = """
            SELECT pickup_latitude, 
            pickup_longitude
            FROM uber_trips
            UNION ALL
            SELECT pickup_latitude, 
            pickup_longitude 
            FROM taxi_trips
            """
    data = pd.read_sql_query(query, engine)
    geoData = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.pickup_longitude, data.pickup_latitude))
    data = g.sjoin(geoData, how="left", predicate="contains")
    
    # count number of rides
    countData = data.groupby(by=["OBJECTID"])["OBJECTID"].count()
    countData.reset_index(inplace=True, drop=True)
    g = g.assign(count=countData)
    
    return g

In [ ]:
def all_hired_trips_heatmap():
    
    fig, ax = plt.subplots(figsize=(15, 10))
    df = get_data_for_visual_4()
    df.plot(column='count', scheme="MaxP", k=8, ax=ax, cmap='Pastel2',legend=True,
            missing_kwds={"color": "lightgrey", "edgecolor": "red", "hatch": "///", "label": "Missing values"},
            legend_kwds={"bbox_to_anchor" :(1.8, 1)})


In [ ]:
all_hired_trips_heatmap()

In [ ]:
heat_map = KeplerGl()
heat_map.add_data(data=get_data_for_visual_4(), name="count")
heat_map

### Visualization 5
### Create a scatter plot that compares tip amount versus distance for Yellow Taxi rides. 

In [ ]:
def scatter_tip_distance(df):
    df = df[df['tip_amount']>0]
    df = df.sample(1000)
    
    fig = plt.figure(figsize=(10, 8))
    plt.scatter(df['distance'], df['tip_amount'])
    
    plt.xlabel('Distance')
    plt.ylabel('Tip amount')
    plt.title('Scatter plot of tip amount and distance')    
    plt.show()

In [ ]:
def get_data_for_visual_5():
    query = """
            SELECT tip_amount,
            distance 
            FROM taxi_trips
            """
    df = pd.read_sql_query(query, engine)
    return df

In [ ]:
df5 = get_data_for_visual_5()
scatter_tip_distance(df5)

### Visualization 6
### Create another scatter plot that compares tip amount versus precipitation amount for Yellow Taxi rides. 

In [ ]:
def scatter_tip_precipitation(df):
    
    df = df[df['tip_amount']>0] # filter tips which are greater than zero 
    fig = plt.figure(figsize=(10, 8))
    plt.scatter(df['HourlyPrecipitation'], df['tip_amount'])
    plt.xlabel('Precipitation')
    plt.ylabel('Tip amount')
    plt.title('Scatter polt of tip amount and precipitation')    
    plt.show()

In [ ]:
def get_data_for_visual_6():
    query_tip = """
                 SELECT tip_amount,
                 strftime('%Y-%m-%d %H', pickup_datetime) AS date
                 FROM taxi_trips
                """
    
    query_rain = """
                 SELECT HourlyPrecipitation, 
                 strftime('%Y-%m-%d %H', DATE) AS date
                 FROM hourly_weather
                 """
    df_tip = pd.read_sql_query(query_tip, engine)
    df_rain = pd.read_sql_query(query_rain, engine)
    df = pd.merge(df_tip, df_rain, on='date')
    df = df.drop('date',axis=1)
    
    return df

In [ ]:
df6 = get_data_for_visual_6()
scatter_tip_precipitation(df6)